### Import required libraries

In [1]:
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import Tool
from datetime import datetime
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor

### Load API Key and Test Model

In [2]:
load_dotenv()

True

In [3]:
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022",  api_key=os.getenv("ANTHROPIC_API_KEY"))
# Test the modle
response = llm.invoke("What is langchain?")
print(response)

content="LangChain is a popular open-source framework designed to help developers build applications powered by large language models (LLMs). Here are its key aspects:\n\n1. Main Purpose:\n- Simplifies the process of creating LLM-powered applications\n- Enables the connection of LLMs with external data sources and environments\n- Provides tools for creating chains of operations with LLMs\n\n2. Key Features:\n- Chains: Combine multiple operations into sequential workflows\n- Agents: Create autonomous AI systems that can plan and execute tasks\n- Memory: Maintain conversation history and context\n- Prompts: Manage and optimize prompts for LLMs\n- Document loaders: Import various document types\n- Vector stores: Enable efficient similarity search\n\n3. Common Use Cases:\n- Question-answering systems\n- Chatbots\n- Document analysis\n- Text summarization\n- Code analysis\n- Data extraction\n\n4. Supported LLMs:\n- OpenAI (GPT models)\n- Google's PaLM\n- Hugging Face models\n- And many othe

### Define Search Tools & Output Parser

In [4]:
search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="search",
    func=search.run,
    description="Search the web for information",
)

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
class ResearchParser(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]

In [7]:
parser = PydanticOutputParser(pydantic_object=ResearchParser)

### Define Chat Prompt template

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a research assistant that will help generate a research paper.
            Answer the user query and use neccessary tools. 
            Wrap the output in this format and provide no other text\n{format_instructions}
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

### Create Agent

In [9]:
tools = [search_tool, wiki_tool]
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=tools
)

### Test the Agent

In [10]:
question = "Tell me about Action Park?"
#
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
raw_response = agent_executor.invoke({"query": question})
structured_response = parser.parse(raw_response.get("output")[0]["text"])
print(structured_response)




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Action Park New Jersey'}`
responded: [{'id': 'toolu_01MbJ7kB3vDtbxdcABgRg5WE', 'input': {}, 'name': 'wikipedia', 'type': 'tool_use', 'index': 0, 'partial_json': '{"query": "Action Park New Jersey"}'}, {'id': 'toolu_01F9dioLbv7gJtX2ndbTybfP', 'input': {}, 'name': 'search', 'type': 'tool_use', 'index': 1, 'partial_json': '{"__arg1": "Action Park New Jersey accidents history"}'}]

Page: Action Park
Summary: Action Park was an amusement and water park located in Vernon Township, N
Invoking: `search` with `Action Park New Jersey accidents history`
responded: [{'id': 'toolu_01MbJ7kB3vDtbxdcABgRg5WE', 'input': {}, 'name': 'wikipedia', 'type': 'tool_use', 'index': 0, 'partial_json': '{"query": "Action Park New Jersey"}'}, {'id': 'toolu_01F9dioLbv7gJtX2ndbTybfP', 'input': {}, 'name': 'search', 'type': 'tool_use', 'index': 1, 'partial_json': '{"__arg1": "Action Park New Jersey accidents history"}'}]

In Vernon Townsh